In [16]:
import os
import nltk
import numpy as np
import pandas as pd

# from pretty import pprint
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

from nltk.corpus.reader import CHILDESCorpusReader
from nltk.downloader import Downloader
nltk_download_dir = Downloader.default_download_dir(Downloader())

Loading BokehJS ...

In [ ]:
corpus_root = os.path.join(nltk_download_dir,
                                 'corpora/CHILDES/Eng-NA')
chi_tagged_sents = []
for idx, corpus_name in enumerate(os.listdir(corpus_root)):
    print "loading:{} - {} of {}".format(corpus_name, idx, len(os.listdir(corpus_root)))
    corpus = CHILDESCorpusReader(root=os.path.join(corpus_root,corpus_name), fileids='.+xml')
    speakers = set()
    for part in corpus.participants():
        for spk in part.keys():
            if spk != "CHI":
                speakers.add(spk)
        
    print "No. of speakers:{}".format(len(speakers))
    stemmed_sents = corpus.sents(fileids=corpus.fileids(),speaker=list(speakers),stem=True)
    tagged_sents = corpus.tagged_sents(fileids=corpus.fileids(),speaker=list(speakers))
    tagged_stemmed_sents = []
    for sent_idx,(tagged_sent,stemmed_sent) in enumerate(zip(tagged_sents,stemmed_sents)):
        tagged_stemmed_sent = []
        for word_idx, (tagged_word, stemmed_word) in enumerate(zip(tagged_sent,stemmed_sent)):
            if "-" in stemmed_word:
                stem, tense = stemmed_word.split("-")
            else:
                stem = stemmed_word
                tense = ""
            tagged_stemmed_sent.append(tagged_word + (stem,tense,)) 
        tagged_stemmed_sents.append(tagged_stemmed_sent)
    chi_tagged_sents += tagged_stemmed_sents
    
#chi_sents = [[y.lower() for y in x] for x in chi_sents]    
print "no of sents:{}, no of words:{}".format(len(chi_sents),len([y for x in chi_sents for y in x]))

loading:Peters - 0 of 48
No. of speakers:25
[('here', 'adv'), ('Weener', 'n:prop')]
> <ipython-input-19-fe5259fcd5de>(18)<module>()
-> for word_idx, (tagged_word, stemmed_word) in enumerate(zip(tagged_sent,stemmed_sent)):
(Pdb) stem,tense
('here', '')
(Pdb) tagged_word
('here', 'adv')
(Pdb) tagged_word + (stem,tense)
('here', 'adv', 'here', '')


In [15]:
import pickle
with open("biling_28_corpora_chd_sents.pkl","w") as op:
    pickle.dump(chi_sents,op)    

In [12]:
corpus_root = os.path.join(nltk_download_dir,
                                 'corpora/CHILDES/Eng-NA/')
chi_sents = []
for idx, corpus_name in enumerate(os.listdir(corpus_root)):
    print "loading:{} - {} of {}".format(corpus_name, idx, len(os.listdir(corpus_root)))
    corpus = CHILDESCorpusReader(root=os.path.join(corpus_root,corpus_name), fileids='.+xml')
    speakers = set()
    for part in corpus.participants():
        for spk in part.keys():
#             print spk
            if spk != "CHI":
                speakers.add(spk)
        
    print "No. of speakers:{}".format(len(speakers))
    chi_sents += list(corpus.sents(fileids=corpus.fileids(),speaker=list(speakers)))
    
print "no of sents:{}, no of words:{}".format(len(chi_sents),len([y for x in chi_sents for y in x]))

loading:Peters - 0 of 48
No. of speakers:25
loading:Kuczaj - 1 of 48
No. of speakers:16
loading:Brent - 2 of 48
No. of speakers:4
loading:MacWhinney - 3 of 48
No. of speakers:30
loading:Warren - 4 of 48
No. of speakers:3
loading:Valian - 5 of 48
No. of speakers:7
loading:Bates - 6 of 48
No. of speakers:4
loading:Evans - 7 of 48
No. of speakers:43
loading:Post - 8 of 48
No. of speakers:4
loading:Higginson - 9 of 48
No. of speakers:9
loading:Bohannon - 10 of 48
No. of speakers:39
loading:VanKleeck - 11 of 48
No. of speakers:1
loading:Cornell - 12 of 48
No. of speakers:37
loading:Braunwald - 13 of 48
No. of speakers:13
loading:Brown - 14 of 48
No. of speakers:31
loading:Clark - 15 of 48
No. of speakers:15
loading:Morisset - 16 of 48
No. of speakers:5
loading:Gathercole - 17 of 48
No. of speakers:33
loading:McCune - 18 of 48
No. of speakers:1
loading:Nelson - 19 of 48
No. of speakers:3
loading:PetersonMcCabe - 20 of 48
No. of speakers:1
loading:Hall - 21 of 48
No. of speakers:16
loading:Ne

In [41]:
print "no of sents:{}, no of words:{}".format(len(chi_sents),len([y for x in chi_sents for y in x]))
chi_sents = [[y.lower() for y in x] for x in chi_sents]
print "no of sents:{}, no of words:{}".format(len(chi_sents),len([y for x in chi_sents for y in x]))

no of sents:2078904, no of words:8916872
no of sents:2078904, no of words:8916872


In [42]:
import pickle
with open("eng_na_48_corpora_non_chi_sents.pkl","w") as op:
    pickle.dump(chi_sents,op)    

In [16]:
sents = pickle.load(open("eng_na_48_corpora_sents.pkl"))

In [14]:
from gensim.models import Word2Vec

In [43]:
model_childes_non_chi = Word2Vec(chi_sents, size=100, window=5, min_count=10, workers=4)

In [44]:
model_childes_non_chi.save("w2v_childes_non_chi_48_corpora_100d")

In [45]:
model_childes = Word2Vec.load("w2v_childes_non_chi_48_corpora_100d")
len(model_childes_non_chi.wv.vocab)

10696

In [17]:
from collections import Counter
words_top_childes =[word for word,_ in Counter(model_childes_non_chi.wv.vocab).most_common(2000)]
words_top_vec_childes = model_childes_non_chi[words_top_childes]

In [18]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_childes_tsne = tsne.fit_transform(words_top_vec_childes)

In [19]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_childes_tsne[:,0],
                                    x2=words_top_childes_tsne[:,1],
                                    names=words_top_childes))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [46]:
model_childes_non_chi.most_similar("ate")

[('eats', 0.6455464363098145),
 ('died', 0.5804065465927124),
 ('drank', 0.5433886051177979),
 ('strawberries', 0.5418684482574463),
 ('counted', 0.5367332696914673),
 ('dropped', 0.5352165699005127),
 ('opened', 0.5203690528869629),
 ('fed', 0.5065531730651855),
 ('tuesday', 0.4864012598991394),
 ('dripped', 0.48536232113838196)]

In [56]:
x="say"
y="sayin"
xx="ask"
# yy="hearing"
print model_childes_non_chi.wv.most_similar(positive=[x, y], negative=[xx])
print "\n"
print model_childes_non_chi.wv.most_similar_cosmul(positive=[x, y], negative=[xx])

[('saying', 0.7501683235168457), ('smashing', 0.5600122809410095), ('doin', 0.5551191568374634), ('doing', 0.5506622791290283), ('banging', 0.5422933101654053), ('hammering', 0.5377984642982483), ('singin', 0.5371065139770508), ('forgetting', 0.535422682762146), ('drooling', 0.5231882929801941), ('scratching', 0.510175883769989)]


[('saying', 1.1412880420684814), ('smashing', 1.0010014772415161), ('banging', 0.9769524931907654), ('doing', 0.9727226495742798), ('hammering', 0.969954788684845), ('doin', 0.9697802662849426), ('gross', 0.9610768556594849), ('forgetting', 0.9568312168121338), ('drooling', 0.9565542340278625), ('singin', 0.9468454122543335)]


In [57]:
x="woman"
y="king"
xx="man"
# yy="hearing"
print model_childes_non_chi.wv.most_similar(positive=[x, y], negative=[xx])
print "\n"
print model_childes_non_chi.wv.most_similar_cosmul(positive=[x, y], negative=[xx])

[('queen', 0.661608099937439), ('stepmother', 0.5044884085655212), ('ghost', 0.499464750289917), ('president', 0.49387654662132263), ('princess', 0.49111342430114746), ('witch', 0.48674657940864563), ('lee', 0.48491913080215454), ('palace', 0.47978657484054565), ('prince', 0.47871941328048706), ('arthur', 0.47593194246292114)]


[('queen', 0.9610638618469238), ('stepmother', 0.8883181214332581), ('lee', 0.8673344254493713), ('tales', 0.8524205684661865), ('named', 0.8489049077033997), ('princess', 0.8421226739883423), ('ghost', 0.8415479063987732), ('godmother', 0.8412274122238159), ('palace', 0.8406979441642761), ('male', 0.8401831388473511)]


# Things to do: 
1. Inspect participants metadata for all speakers
2. Min edit distance
3. Restrict to only plurals, past-tenses, present-continuous
4. Average the difference vectors between pairs, and apply the transformation, to find the vector that is most similar to the resultant vector